In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process

C:\Users\chris\.virtualenvs\Maskininlarning-Christopher-Seander-nLYLwDGA\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
movies = "ml-latest-small/movies.csv"
ratings = "ml-latest-small/ratings.csv"

df_movies = pd.read_csv(movies, usecols=["movieId", "title"], dtype={"movieId" : "int32", "title" : "str"})
df_ratings = pd.read_csv(ratings, usecols=["userId", "movieId", "rating"], dtype={"userId" : "int32", "movieId" : "int32", "rating" : "float"})

In [9]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int32  
 1   movieId  100836 non-null  int32  
 2   rating   100836 non-null  float64
dtypes: float64(1), int32(2)
memory usage: 1.5 MB


In [3]:
# Sparse Matrix
#             Users
#           [4, 4, 5]   A
# Movives   [3, 3, 4]   B  ==  Cos(A,B) => 0.95
#           [3, 2, 1]

movies_users = df_ratings.pivot(index="movieId", columns="userId", values="rating").fillna(0)
mat_movies_users = csr_matrix(movies_users.values)

In [4]:
# Euclidean Distance
# Manhattan Distance
# Minkowski Distance
# Cosine Similarity

model_KNN = NearestNeighbors(metric="cosine", algorithm="brute", n_neighbors=20)

In [5]:
model_KNN.fit(mat_movies_users)
model_KNN.kneighbors(mat_movies_users[0])

(array([[0.        , 0.42739874, 0.4343632 , 0.43573831, 0.44261183,
         0.45290409, 0.45885465, 0.4589107 , 0.46108723, 0.46583124,
         0.46961865, 0.4720232 , 0.47214083, 0.47967548, 0.48196726,
         0.48581831, 0.48775438, 0.49068283, 0.49140738, 0.49145545]]),
 array([[   0, 2353,  418,  615,  224,  314,  322,  910,  546,  963,  968,
         3189,  506,  123,  257,  897,  815, 1182,   31,  277]],
       dtype=int64))

In [6]:
# Recommender(movie_name) => List of Movies recommender

def recommender(movie_name, data, model, n_recommendations):
    model.fit(data)
    index = process.extractOne(movie_name, df_movies["title"])[2]
    print(f"Movie Selected: {df_movies['title'][index]}")
    print('Searching for recommendations...')

    distances, indices = model.kneighbors(data[index], n_neighbors = n_recommendations)
    for i in indices:
        print(df_movies["title"][i].where(i != index))


recommender("iron man", mat_movies_users, model_KNN, 5)

Movie Selected: Iron Man (2008)
Searching for recommendations...
6743                        NaN
7197              Garage (2007)
7195    Merry Madagascar (2009)
7354         A-Team, The (2010)
6726     Superhero Movie (2008)
Name: title, dtype: object


In [ ]:
movie_users = asd.pivot(index="movieId", columns="userId", values="rating").fillna(0)
testing = csr_matrix(movie_users.values)